In [4]:
from seligator.common.params import MetadataEncoding, Seq2VecEncoderType, BasisVectorConfiguration
from seligator.simple_demo import prepare_model, train_and_get
from seligator.tests.evaluate import *
from seligator.models.siamese import SiameseClassifier
from seligator.models.classifier import FeatureEmbeddingClassifier
import os
import json
import logging

logger = logging.getLogger()
logger.setLevel(logging.WARNING)

def get_json_fn(fn):
    return f"results-{fn}.json"

def already_done(fn):
    return os.path.isfile(get_json_fn(fn))

def save_json(fn, obj):
    with open(get_json_fn(fn), "w"):
        json.dump(obj, fn)


# Run tests on Vectors Categories
def get_kwargs():
    METADATA_CATS = ("Century", "Textgroup", "WrittenType", "CitationTypes")
    BVC = BasisVectorConfiguration(
        categories=METADATA_CATS
    )
    return dict(
        input_features=("lemma_char", "lemma", "case", "numb", "gend", "mood", "tense", "voice", "person", "deg"),
        seq2vec_encoder_type=Seq2VecEncoderType.LSTM,
        basis_vector_configuration=BVC,
        agglomerate_msd=False,
        reader_kwargs={
            "batch_size": 4, 
            "metadata_encoding": MetadataEncoding.AS_CATEGORICAL,
            "metadata_tokens_categories": METADATA_CATS
        },
        model_embedding_kwargs=dict(
            keep_all_vocab=True,
            pretrained_embeddings={
                # "token": "~/Downloads/latin.embeddings",
            #    "token": "~/dev/these/notebooks/4 - Detection/data/embs_models/model.token.word2vec.kv",
            #    "lemma": "~/dev/these/notebooks/4 - Detection/data/embs_models/model.lemma.word2vec.kv.header"
            },
            trainable_embeddings={"token": False, "lemma": False},
            pretrained_emb_dims={"token": 200, "lemma": 200}
        ),
        #batches_per_epoch=100,
        model_class=FeatureEmbeddingClassifier,
        use_bert_highway=True
    )

    # 
#        additional_model_kwargs={
#            "metadata_linear": False,
#        }
    
def get_train_and_get_kwargs():
    return dict(patience=2, num_epochs=20, lr=5e-4, optimizer="AdamW")

def jqs(data):
    return "-".join(sorted(list(data)))

def get_filename(params, prefix = "model"):
    remaped = []
    for key in sorted(list(params.keys())):
        if isinstance(params[key], str) and not params[key]:
            continue
        if isinstance(params[key], str) and "-" in params[key]:
            remaped.append(f"{key}-"+"".join([
                "".join([
                    subv[:3].lower().capitalize()
                    for subv in v.split("_")
                ])
                for v in params[key].split("-")
            ]))
        else:
            remaped.append(f"{key}-{params[key]}")
    return prefix+"--"+"__".join(remaped)

def merge(source, destination):
    """
    run me with nosetests --with-doctest file.py

    >>> a = { 'first' : { 'all_rows' : { 'pass' : 'dog', 'number' : '1' } } }
    >>> b = { 'first' : { 'all_rows' : { 'fail' : 'cat', 'number' : '5' } } }
    >>> merge(b, a) == { 'first' : { 'all_rows' : { 'pass' : 'dog', 'fail' : 'cat', 'number' : '5' } } }
    True
    """
    for key, value in source.items():
        if isinstance(value, dict):
            # get node or create one
            node = destination.setdefault(key, {})
            merge(value, node)
        else:
            destination[key] = value

    return destination

def run_and_save(prepare_model_kwargs, train_kwargs, model_name_prefix: str = "model"):
    main_params = {
        "Features": jqs(prepare_model_kwargs["input_features"]),
        "Metadata": jqs(prepare_model_kwargs["reader_kwargs"]["metadata_tokens_categories"]),
        "MetadataEncoding": prepare_model_kwargs["reader_kwargs"]["metadata_encoding"].name,
        "Pretrain": jqs(prepare_model_kwargs["model_embedding_kwargs"]["pretrained_embeddings"].keys()),
        "Trainable": jqs([
            key 
            for key in prepare_model_kwargs["model_embedding_kwargs"]["trainable_embeddings"]
            if key in prepare_model_kwargs["model_embedding_kwargs"]["pretrained_embeddings"] and
            prepare_model_kwargs["model_embedding_kwargs"]["trainable_embeddings"][key] is True
        ]),
        "AgglomerateMSD": prepare_model_kwargs["agglomerate_msd"],
        "EncoderType": prepare_model_kwargs["seq2vec_encoder_type"].name,
        **prepare_model_kwargs.get("additional_model_kwargs", {}),
        "Model": prepare_model_kwargs["model_class"].__name__.replace("Classifier", "")
    }
    if main_params["Model"] == "Siamese":
        main_params["Highway"] = prepare_model_kwargs.get("use_bert_highway", False)
        main_params["BpE"] = prepare_model_kwargs.get("batches_per_epoch", None)
        
    fn = get_filename(main_params, model_name_prefix)
    
    if already_done(fn):
        return {}
    
    model, reader, train, dev = prepare_model(**prepare_model_kwargs)
    model = train_and_get(model, train, dev, **train_kwargs)
    data, img = run_tests(
        "dataset/split/test.txt",
        dataset_reader=reader, model=model, dump=f"test.{fn}.csv"
    )
    out = {
        fn: {
            **{x:v for x, v in data.items() if isinstance(v, float)},
            **main_params,
            **train_kwargs
        }
    }
    save_json(fn, out)
    return out

def get_siamese():
    siamese = get_kwargs()
    siamese["model_class"] = SiameseClassifier
    siamese["batches_per_epoch"] = 20
    siamese_train_kwargs = get_train_and_get_kwargs()
    siamese_train_kwargs["num_epochs"] = siamese_train_kwargs["num_epochs"] * int(1351 // siamese["batches_per_epoch"])
    siamese_train_kwargs["patience"] = siamese_train_kwargs["patience"] * int(1351 // siamese["batches_per_epoch"])
    return siamese, siamese_train_kwargs

def get_classic():
    return get_kwargs(), get_train_and_get_kwargs()

RUNS = {}

In [6]:
def do_runs(get_kw, has_linear=True):
    Changes = [
        # Use raw Bert
        ({"input_features": ("token_subword", )}, {}),
        # Use raw Bert No Highway
        ({"input_features": ("token_subword", ), "use_bert_highway": True}, {}),
        # Use raw Bert + Lemma
        ({
            "input_features": ("token_subword", "lemma", "lemma_char"),
            "seq2vec_encoder_type": Seq2VecEncoderType.HAN,
            "use_bert_highway": False
        }, {}),
        ({}, {}),
        # Raw Features + MSD + Vanilla LSTM
        ({"agglomerate_msd": True}, {}),
        # Raw Features + MSD + HAN
        ({"agglomerate_msd": True, "seq2vec_encoder_type": Seq2VecEncoderType.HAN}, {}),
        # Raw Features + MSD + Enriched LSTM
        ({"agglomerate_msd": True, "seq2vec_encoder_type": Seq2VecEncoderType.MetadataLSTM}, {}),
        # Raw Features + MSD + Enriched Attention
        ({"agglomerate_msd": True, "seq2vec_encoder_type": Seq2VecEncoderType.MetadataAttentionPooling}, {}),
        # Raw Features + MSD + Attention
        ({"agglomerate_msd": True, "seq2vec_encoder_type": Seq2VecEncoderType.AttentionPooling}, {}),
        # Now we use Metadata Tokens !
        # Raw Features + MSD + Attention
        ({
            "reader_kwargs": {"metadata_encoding": MetadataEncoding.AS_TOKEN},
            "agglomerate_msd": True, "seq2vec_encoder_type": Seq2VecEncoderType.AttentionPooling}, {}),
        # Raw Features + MSD + Attention
        ({
            "reader_kwargs": {"metadata_encoding": MetadataEncoding.AS_TOKEN},
            "agglomerate_msd": True, "seq2vec_encoder_type": Seq2VecEncoderType.HAN}, {}),
        
    ]
    if has_linear:
        Changes = Changes + [
            ({"additional_model_kwargs": { "metadata_linear": True}}, {})
        ]
    for model_kw, train_kw in Changes:
        defaults_model, default_trains = get_kw()
        defaults_model = merge(defaults_model, model_kw)
        train_kw = merge(default_trains, train_kw)
        RUNS.update(run_and_save(model_kw, train_kw))

# Siamese Networks

In [ ]:
do_runs(get_siamese)

/home/thibault/dev/est-lascivuum-non-est/env/lib/python3.8/site-packages/allennlp/data/token_indexers/token_characters_indexer.py:55: UserWarning: You are using the default value (0) of `min_padding_length`, which can cause some subtle bugs (more info see https://github.com/allenai/allennlp/issues/1954). Strongly recommend to set a value, usually the maximum size of the convolutional layer size when using CnnEncoder.
  warnings.warn(


building vocab:   0%|          | 0/5425 [00:00<?, ?it/s]

---> Epochs:   1340
---> Patience: 134


  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
RUNS

In [ ]:
best, best_key = 0, None
sorts = sorted([(key, val["accuracy"]) for key, val in RUNS.items()], key=lambda x: x[1])
for key in sor